<div class="frontmatter text-center">
<h1>Introduction to Data Science and Programming</h1>
<h2>Class 14: Data relationships</h2>
<h3>IT University of Copenhagen, Fall 2019</h3>
<h3>Instructor: Michael Szell</h3>
</div>

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline  
import scipy.stats as stats

## Loading a mixed variable type data set

In [ ]:
# Data set downloaded from: https://www.kaggle.com/mirichoi0218/insurance/downloads/insurance.zip/1
!head files/insurance.csv

The data set contains demographic information about individuals, where they live (region), whether they smoke, and how much they pay for health insurance (charges).

We read the data as structured arrays using `numpy.genfromtxt` with `dtype=None`. `names=True` ensures we can access single columns using the header. More info: https://numpy.org/devdocs/user/basics.io.genfromtxt.html

In [ ]:
datamixed = np.genfromtxt('files/insurance.csv', delimiter=',', dtype=None, names=True, encoding='utf8')
print(datamixed)
print(datamixed['smoker'])

In [ ]:
fig = plt.figure(figsize=(5, 4))
axes = fig.add_axes([0, 0, 1, 1])

# Plot data
axes.scatter(datamixed['bmi'], datamixed['charges'], label='Data', edgecolors='w');
axes.legend(loc='upper left'); axes.set_xlabel('BMI'); axes.set_ylabel('Charges'); axes.set_title('Scatter plot');

In [ ]:
fig = plt.figure(figsize=(5, 4))
axes = fig.add_axes([0, 0, 1, 1])

# Plot data
axes.scatter(datamixed['bmi'], datamixed['charges'], label='Data', edgecolors='w');

# Plot fit
slope, intercept, r_value, _, _ = stats.linregress(datamixed['bmi'], datamixed['charges'])
line = slope * datamixed['bmi'] + intercept
axes.plot(datamixed['bmi'], line, linewidth=2.5, color='k', label='Fit')

axes.text(45, 24000, "r = " + str(round(r_value, 3)))
axes.legend(loc='upper left'); axes.set_xlabel('BMI'); axes.set_ylabel('Charges'); axes.set_title('Scatter plot');

The data points relate in a very weird way. Some stay on the bottom, some go up, and there is some "bottleneck" at BMI=30. Sure, we can fit a linear trend, but is it meaningful? We have to be careful.

Maybe adding information from a categorical variable clears things up:

In [ ]:
masksmoker = (datamixed['smoker'] == 'yes')  # Easy indexing: The power of numpy

fig = plt.figure(figsize=(5, 4))
axes = fig.add_axes([0, 0, 1, 1])

# Plot data
axes.scatter(datamixed['bmi'][~masksmoker], datamixed['charges'][~masksmoker], color='orange', label="Data nonsmoker", edgecolors='w');
axes.scatter(datamixed['bmi'][masksmoker], datamixed['charges'][masksmoker], color='k', label="Data smoker", edgecolors='w');

axes.legend(); axes.set_xlabel('BMI'); axes.set_ylabel('Charges'); axes.set_title('Scatter plot');

Aha! We have two sub-groups with different relations: Smokers and non-smokers are treated differently for insurance purposes. Next, let us do the linear regression on each of the sub-groups:

In [ ]:
fig = plt.figure(figsize=(5, 4))
axes = fig.add_axes([0, 0, 1, 1])

# Plot data
axes.scatter(datamixed['bmi'][~masksmoker], datamixed['charges'][~masksmoker], color='orange', label="Data nonsmoker", edgecolors='w');
axes.scatter(datamixed['bmi'][masksmoker], datamixed['charges'][masksmoker], color='k', label="Data smoker", edgecolors='w');

# Plot fits
slope, intercept, r_value, _, _ = stats.linregress(datamixed['bmi'][~masksmoker], datamixed['charges'][~masksmoker])
axes.plot(datamixed['bmi'][~masksmoker], slope * datamixed['bmi'][~masksmoker] + intercept, linewidth=2.5, color='orange', label='Fit nonsmoker')
axes.text(45, 13000, "r = " + str(round(r_value, 3)))
slope, intercept, r_value, _, _ = stats.linregress(datamixed['bmi'][masksmoker], datamixed['charges'][masksmoker])
axes.plot(datamixed['bmi'][masksmoker], slope * datamixed['bmi'][masksmoker] + intercept, linewidth=2.5, color='k', label='Fit smoker')
axes.text(42, 58000, "r = " + str(round(r_value, 3)))

axes.legend(); axes.set_xlabel('BMI'); axes.set_ylabel('Charges'); axes.set_title('Scatter plot');

Our conclusion, if we stop here:
* **For non-smokers**: There is a very weak linear relation between BMI and charges. Apparently the insurance does not care too much about your BMI if you do not smoke
* **For smokers**: There seem to be 2 sub-groups of smokers treated differently by the insurance: Those with BMI below 30 and those with BMI above 30. The linear regression through both these groups shows that High-BMI smokers pay higher charges. This should not be the end of the analysis, however: Since we have two sub-groups, we have to question whether a linear regression through them is justified.

## Relating categorical and quantitative data

In [ ]:
fig = plt.figure(figsize=(4, 3))
axes = fig.add_axes([0, 0, 1, 1]) # left, bottom, width, height (range 0 to 1)
axes.boxplot([datamixed['charges'][masksmoker], datamixed['charges'][~masksmoker]]);

axes.set_ylabel('Charges')
axes.set_xticklabels(['Smokers', 'Nonsmokers']);

In [ ]:
fig = plt.figure(figsize=(4, 3))
axes = fig.add_axes([0, 0, 1, 1]) # left, bottom, width, height (range 0 to 1)
axes.boxplot([datamixed['charges'][datamixed['region'] == 'southwest'],
              datamixed['charges'][datamixed['region'] == 'southeast'], 
              datamixed['charges'][datamixed['region'] == 'northwest'], 
              datamixed['charges'][datamixed['region'] == 'northeast']]);

axes.set_ylabel('Charges')
axes.set_xticklabels(['South west', 'South east', 'North West', 'North east']);